In [1]:

# -*- coding: utf-8 -*-
'''
Stock Prices Prediction Project with Neptune.ai
'''
import os

###### Import all the packages for analysis
import pandas as pd
import numpy as np

# for reproducibility of our results
np.random.seed(42)

from datetime import date
from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential, Model
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate

import tensorflow as tf 
tf.random.set_seed(42)

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json

In [2]:
# Sort DataFrame by date

traindata = pd.read_csv('/content/drive/MyDrive/JP_Morgan/Train.csv')

In [3]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 15000 non-null  object 
 1   Stock                15000 non-null  object 
 2   liabilities          15000 non-null  float64
 3   equity               15000 non-null  float64
 4   total_assets         15000 non-null  float64
 5   current_assets       15000 non-null  float64
 6   current_liabilities  15000 non-null  float64
 7   total_revenue        15000 non-null  float64
 8   net_income           15000 non-null  float64
 9   dividend             15000 non-null  float64
 10  shares_outstanding   15000 non-null  int64  
 11  price                15000 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 1.4+ MB


In [4]:
def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return dt.datetime(year=year, month=month, day=day)

datetime_object = str_to_datetime('1986-03-19')
datetime_object

datetime.datetime(1986, 3, 19, 0, 0)

In [5]:
traindata.head()

,Date,Stock,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price
0,2006-07-31,Stock 1,625.898686,925.086518,1550.985204,675.789905,555.931329,548.488451,251.282553,68.184030,1040831,100.000000
1,2006-08-31,Stock 1,407.387467,903.249093,1310.636561,1023.110983,1295.186240,513.446432,207.576940,42.232672,1046649,101.924169
2,2006-09-29,Stock 1,464.069891,602.668832,1066.738722,709.711673,598.719338,503.519871,133.719700,28.835722,1094158,103.166191
3,2006-10-31,Stock 1,855.854937,391.375369,1247.230306,649.979564,733.594339,596.006397,-76.881818,0.000000,1018574,104.478956
4,2006-11-30,Stock 1,453.954969,956.490566,1410.445534,1115.594551,1152.274711,965.939247,-309.204301,0.000000,1036293,104.160907


In [6]:
traindata

,Date,Stock,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price
0,2006-07-31,Stock 1,625.898686,925.086518,1550.985204,675.789905,555.931329,548.488451,251.282553,68.184030,1040831,100.000000
1,2006-08-31,Stock 1,407.387467,903.249093,1310.636561,1023.110983,1295.186240,513.446432,207.576940,42.232672,1046649,101.924169
2,2006-09-29,Stock 1,464.069891,602.668832,1066.738722,709.711673,598.719338,503.519871,133.719700,28.835722,1094158,103.166191
3,2006-10-31,Stock 1,855.854937,391.375369,1247.230306,649.979564,733.594339,596.006397,-76.881818,0.000000,1018574,104.478956
4,2006-11-30,Stock 1,453.954969,956.490566,1410.445534,1115.594551,1152.274711,965.939247,-309.204301,0.000000,1036293,104.160907
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2018-08-31,Stock 100,810.402398,953.465125,1763.867523,1014.718528,957.998121,675.953821,102.346000,21.785595,1016713,101.747952
14996,2018-09-28,Stock 100,811.620957,443.679899,1255.300856,392.059390,365.958118,815.913721,-301.200669,0.000000,1042638,102.039499
14997,2018-10-31,Stock 100,739.371050,423.172140,1162.543189,478.844995,396.603049,550.561664,227.299298,56.788344,1077825,101.126752
14998,2018-11-30,Stock 100,741.898496,286.891422,1028.789919,347.793349,358.229637,176.189978,19.405005,5.183536,1003161,102.020776


In [7]:
traindata['current_Ratio'] = traindata['current_assets']/traindata['current_liabilities']
traindata['dividend_Payout'] = traindata['dividend']/traindata['net_income']
traindata['PE_Ratio'] = traindata['price']/(traindata['net_income']/traindata['shares_outstanding'])
traindata['proprietary_ratio'] = traindata['equity']/traindata['total_assets']

traindata

,Date,Stock,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price,current_Ratio,dividend_Payout,PE_Ratio,proprietary_ratio
0,2006-07-31,Stock 1,625.898686,925.086518,1550.985204,675.789905,555.931329,548.488451,251.282553,68.184030,1040831,100.000000,1.215600,0.271344,4.142074e+05,0.596451
1,2006-08-31,Stock 1,407.387467,903.249093,1310.636561,1023.110983,1295.186240,513.446432,207.576940,42.232672,1046649,101.924169,0.789933,0.203456,5.139243e+05,0.689168
2,2006-09-29,Stock 1,464.069891,602.668832,1066.738722,709.711673,598.719338,503.519871,133.719700,28.835722,1094158,103.166191,1.185383,0.215643,8.441547e+05,0.564964
3,2006-10-31,Stock 1,855.854937,391.375369,1247.230306,649.979564,733.594339,596.006397,-76.881818,0.000000,1018574,104.478956,0.886020,-0.000000,-1.384197e+06,0.313796
4,2006-11-30,Stock 1,453.954969,956.490566,1410.445534,1115.594551,1152.274711,965.939247,-309.204301,0.000000,1036293,104.160907,0.968167,-0.000000,-3.490935e+05,0.678148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2018-08-31,Stock 100,810.402398,953.465125,1763.867523,1014.718528,957.998121,675.953821,102.346000,21.785595,1016713,101.747952,1.059207,0.212862,1.010772e+06,0.540554
14996,2018-09-28,Stock 100,811.620957,443.679899,1255.300856,392.059390,365.958118,815.913721,-301.200669,0.000000,1042638,102.039499,1.071323,-0.000000,-3.532205e+05,0.353445
14997,2018-10-31,Stock 100,739.371050,423.172140,1162.543189,478.844995,396.603049,550.561664,227.299298,56.788344,1077825,101.126752,1.207366,0.249840,4.795305e+05,0.364006
14998,2018-11-30,Stock 100,741.898496,286.891422,1028.789919,347.793349,358.229637,176.189978,19.405005,5.183536,1003161,102.020776,0.970867,0.267124,5.274065e+06,0.278863


In [8]:
traindata = traindata[['Date','Stock','current_liabilities','total_revenue','current_Ratio','dividend_Payout','PE_Ratio','proprietary_ratio','price']]
traindata

,Date,Stock,current_liabilities,total_revenue,current_Ratio,dividend_Payout,PE_Ratio,proprietary_ratio,price
0,2006-07-31,Stock 1,555.931329,548.488451,1.215600,0.271344,4.142074e+05,0.596451,100.000000
1,2006-08-31,Stock 1,1295.186240,513.446432,0.789933,0.203456,5.139243e+05,0.689168,101.924169
2,2006-09-29,Stock 1,598.719338,503.519871,1.185383,0.215643,8.441547e+05,0.564964,103.166191
3,2006-10-31,Stock 1,733.594339,596.006397,0.886020,-0.000000,-1.384197e+06,0.313796,104.478956
4,2006-11-30,Stock 1,1152.274711,965.939247,0.968167,-0.000000,-3.490935e+05,0.678148,104.160907
...,...,...,...,...,...,...,...,...,...
14995,2018-08-31,Stock 100,957.998121,675.953821,1.059207,0.212862,1.010772e+06,0.540554,101.747952
14996,2018-09-28,Stock 100,365.958118,815.913721,1.071323,-0.000000,-3.532205e+05,0.353445,102.039499
14997,2018-10-31,Stock 100,396.603049,550.561664,1.207366,0.249840,4.795305e+05,0.364006,101.126752
14998,2018-11-30,Stock 100,358.229637,176.189978,0.970867,0.267124,5.274065e+06,0.278863,102.020776


In [9]:
traindata['Date'] = traindata['Date'].apply(str_to_datetime)

<ipython-input-9-f5802138652d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stockprices['Date'] = stockprices['Date'].apply(str_to_datetime)


In [10]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 15000 non-null  datetime64[ns]
 1   Stock                15000 non-null  object        
 2   current_liabilities  15000 non-null  float64       
 3   total_revenue        15000 non-null  float64       
 4   current_Ratio        15000 non-null  float64       
 5   dividend_Payout      15000 non-null  float64       
 6   PE_Ratio             15000 non-null  float64       
 7   proprietary_ratio    15000 non-null  float64       
 8   price                15000 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 1.0+ MB


In [11]:
traindata = traindata.set_index('Date')
traindata

,Stock,current_liabilities,total_revenue,current_Ratio,dividend_Payout,PE_Ratio,proprietary_ratio,price
Date,,,,,,,,
2006-07-31,Stock 1,555.931329,548.488451,1.215600,0.271344,4.142074e+05,0.596451,100.000000
2006-08-31,Stock 1,1295.186240,513.446432,0.789933,0.203456,5.139243e+05,0.689168,101.924169
2006-09-29,Stock 1,598.719338,503.519871,1.185383,0.215643,8.441547e+05,0.564964,103.166191
2006-10-31,Stock 1,733.594339,596.006397,0.886020,-0.000000,-1.384197e+06,0.313796,104.478956
2006-11-30,Stock 1,1152.274711,965.939247,0.968167,-0.000000,-3.490935e+05,0.678148,104.160907
...,...,...,...,...,...,...,...,...
2018-08-31,Stock 100,957.998121,675.953821,1.059207,0.212862,1.010772e+06,0.540554,101.747952
2018-09-28,Stock 100,365.958118,815.913721,1.071323,-0.000000,-3.532205e+05,0.353445,102.039499
2018-10-31,Stock 100,396.603049,550.561664,1.207366,0.249840,4.795305e+05,0.364006,101.126752


In [12]:
#### Define helper functions to calculate the metrics RMSE and MAPE ####
def calculate_rmse(y_true, y_pred):
    """
    Calculate the Root Mean Squared Error (RMSE)  
    """
    rmse = np.sqrt(np.mean((y_true-y_pred)**2))                   
    return rmse

### The effectiveness of prediction method is measured in terms of the Mean Absolute Percentage Error (MAPE) and RMSE
def calculate_mape(y_true, y_pred): 
    """
    Calculate the Mean Absolute Percentage Error (MAPE) %
    """
    y_pred, y_true = np.array(y_pred), np.array(y_true)    
    mape = np.mean(np.abs((y_true-y_pred) / y_true))*100    
    return mape

In [13]:
## Split the time-series data into training seq X and output value Y
def extract_seqX_outcomeY(data, N, offset):
    """
    Split time-series into training sequence X and outcome value Y
    Args:
        data - dataset 
        N - window size, e.g., 60 for 60 days 
        offset - position to start the split
    """
    X, y = [], []
    
    for i in range(offset, len(data)):
        X.append(data[i-N:i])
        y.append(data[i])
    
    return np.array(X), np.array(y)

In [15]:
#### Train-testdata split for time-series ####
# testdata_ratio = 0.2
# training_ratio = 1 - testdata_ratio

# train_size = int(training_ratio * len(traindata))
# testdata_size = int(testdata_ratio * len(traindata))

# print("train_size: " + str(train_size))
# print("testdata_size: " + str(testdata_size))

# train = traindata[:train_size][['Date', 'Close']]
# testdata = traindata[train_size:][['Date', 'Close']]

# print(len(traindata));

# train = traindata[2600:3050][['Date', 'Close']]
# testdata = traindata[3050:][['Date', 'Close']]

In [16]:
###================= simple MA


### For meduim-term trading 
def plot_stock_trend(var, cur_title, traindata=traindata, logmodelName='Simple MA'):
    ax = traindata[['price', var,'200day']].plot(figsize=(20, 10))
    plt.grid(False)
    plt.title(cur_title)
    plt.axis('tight')
    plt.ylabel('Stock Price ($)')
    
        
def calculate_perf_metrics(var, logmodelName='Simple MA'):
    ### RMSE 
    rmse = calculate_rmse(np.array(traindata[:]['price']), np.array(traindata[:][var]))
    ### MAPE 
    mape = calculate_mape(np.array(traindata[:]['price']), np.array(traindata[:][var]))
    
    
    return rmse, mape


In [17]:

# 20 days to represent the 22 trading days in a month
window_size = 50
CURRENT_MODEL = 'LSTM'
if CURRENT_MODEL == 'LSTM':
    
    layer_units, optimizer = 50, 'adam' 
    cur_epochs = 100
    cur_batch_size = 50
    
    cur_LSTM_pars = {'units': layer_units, 
                     'optimizer': optimizer, 
                     'batch_size': cur_batch_size, 
                     'epochs': cur_epochs
                     }
      
    ## use the past N stock prices for training to predict the N+1th closing price
   
    # scale 
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(traindata[['price']])
    scaled_data_train = scaled_data[:traindata.shape[0]]
    
    X_train, y_train = extract_seqX_outcomeY(scaled_data_train, window_size, window_size)
    
    ### Build a LSTM model and log model summary to Neptune ###    
    def Run_LSTM(X_train, layer_units=50):     
        inp = Input(shape=(X_train.shape[1], 1))
        
        x = LSTM(units=layer_units, return_sequences=True)(inp)
        x = LSTM(units=layer_units)(x)
        out = Dense(1, activation='linear')(x)
        model = Model(inp, out)
        
        # Compile the LSTM neural net
        model.compile(loss = 'mean_squared_error', optimizer = 'adam')
        return model   
    
    model = Run_LSTM(X_train, layer_units=layer_units)
    
    history = model.fit(X_train, y_train, epochs=cur_epochs, batch_size=cur_batch_size, 
                        verbose=1, validation_split=0.1, shuffle=True)
    
    # predict stock prices using past window_size stock prices

    # testdata = pd.read_csv('/content/drive/MyDrive/JP_Morgan/testdata_x.csv')


    # def preprocess_testdatadat(data=traindata, scaler=scaler, window_size=window_size, testdata=testdata):    
    #     raw = data['Close'][len(data) - len(testdata) - window_size:].values
    #     raw = raw.reshape(-1,1)
    #     raw = scaler.transform(raw)
        
    #     X_testdata = []
    #     for i in range(window_size, raw.shape[0]):
    #         X_testdata.append(raw[i-window_size:i, 0])
            
    #     X_testdata = np.array(X_testdata)
        
    #     X_testdata = np.reshape(X_testdata, (X_testdata.shape[0], X_testdata.shape[1], 1))
    #     return X_testdata
    
    # X_testdata = preprocess_testdatadat()
    
    # predicted_price_ = model.predict(X_testdata)
    # predicted_price = scaler.inverse_transform(predicted_price_)
    
    # Plot predicted price vs actual closing price 
    # testdata['Predictions_lstm'] = predicted_price   
    
    # # Evaluate performance
    # rmse_lstm = calculate_rmse(np.array(testdata['Close']), np.array(testdata['Predictions_lstm']))
    # mape_lstm = calculate_mape(np.array(testdata['Close']), np.array(testdata['Predictions_lstm']))
  
    # ### Plot prediction and true trends and log to Neptune         
    # def plot_stock_trend_lstm(train, testdata):        
    #     fig = plt.figure(figsize = (20,10))
    #     plt.plot(train['Date'], train['Close'], label = 'Train Closing Price')
    #     plt.plot(testdata['Date'], testdata['Close'], label = 'testdata Closing Price')
    #     plt.plot(testdata['Date'], testdata['Predictions_lstm'], label = 'Predicted Closing Price')
    #     plt.title('LSTM Model')
    #     plt.xlabel('Date')
    #     plt.ylabel('Stock Price ($)')
    #     plt.legend(loc="upper left")    
    # plot_stock_trend_lstm(train, testdata)
    

Epoch 1/100
270/270 [==============================] - 14s 17ms/step - loss: 0.1010 - val_loss: 0.0410
Epoch 2/100
270/270 [==============================] - 2s 7ms/step - loss: 0.0303 - val_loss: 0.0233
Epoch 3/100
270/270 [==============================] - 2s 7ms/step - loss: 0.0218 - val_loss: 0.0240
Epoch 4/100
270/270 [==============================] - 2s 8ms/step - loss: 0.0213 - val_loss: 0.0218
Epoch 5/100
270/270 [==============================] - 2s 8ms/step - loss: 0.0213 - val_loss: 0.0234
Epoch 6/100
270/270 [==============================] - 3s 9ms/step - loss: 0.0208 - val_loss: 0.0222
Epoch 7/100
270/270 [==============================] - 3s 10ms/step - loss: 0.0206 - val_loss: 0.0232
Epoch 8/100
270/270 [==============================] - 2s 8ms/step - loss: 0.0205 - val_loss: 0.0217
Epoch 9/100
270/270 [==============================] - 2s 7ms/step - loss: 0.0205 - val_loss: 0.0216
Epoch 10/100
270/270 [==============================] - 2s 7ms/step - loss: 0.0204 - val

In [20]:
testdata = pd.read_csv('/content/drive/MyDrive/JP_Morgan/testdata_x.csv')
testdata['current_Ratio'] = testdata['current_assets']/testdata['current_liabilities']
testdata['dividend_Payout'] = testdata['dividend']/testdata['net_income']
# testdata['PE_Ratio'] = testdata['price']/(testdata['net_income']/testdata['shares_outstanding'])
testdata['proprietary_ratio'] = testdata['equity']/testdata['total_assets']

testdata = testdata[['Date','Stock','current_liabilities','total_revenue','current_Ratio','dividend_Payout','proprietary_ratio']]
testdata['Date'] = testdata['Date'].apply(str_to_datetime)
testdata = testdata.set_index('Date')
testdata

,Stock,current_liabilities,total_revenue,current_Ratio,dividend_Payout,proprietary_ratio
Date,,,,,,
2019-01-31,Stock 1,834.292436,101.282767,0.881437,0.280577,0.346465
2019-02-28,Stock 1,447.906629,119.483578,1.031850,0.221432,0.643166
2019-03-29,Stock 1,683.167288,482.381171,0.774725,-0.000000,0.428754
2019-04-30,Stock 1,350.779135,570.934617,1.176724,0.254717,0.625610
2019-05-31,Stock 1,892.693784,277.522337,0.899257,-0.000000,0.711293
...,...,...,...,...,...,...
2022-10-31,Stock 100,312.827547,150.677789,1.112762,-0.000000,0.518722
2022-11-30,Stock 100,892.471680,529.905187,1.025366,0.208471,0.597438
2022-12-30,Stock 100,420.021023,963.856236,0.841474,-0.000000,0.642605


In [22]:
def preprocess_testdatadat(data=traindata, scaler=scaler, window_size=window_size, testdata=testdata):    
    raw = data['price'][len(data) - len(testdata) - window_size:].values
    raw = raw.reshape(-1,1)
    raw = scaler.transform(raw)
    
    X_testdata = []
    for i in range(window_size, raw.shape[0]):
        X_testdata.append(raw[i-window_size:i, 0])
        
    X_testdata = np.array(X_testdata)
    
    X_testdata = np.reshape(X_testdata, (X_testdata.shape[0], X_testdata.shape[1], 1))
    return X_testdata

X_testdata = preprocess_testdatadat()

predicted_price_ = model.predict(X_testdata)
predicted_price = scaler.inverse_transform(predicted_price_)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


157/157 [==============================] - 1s 4ms/step


In [28]:
# Plot predicted price vs actual closing price 
testdata['Predictions_lstm'] = predicted_price   
testdata

# Evaluate performance
# rmse_lstm = calculate_rmse(np.array(testdata['Predictions_lstm']))
# mape_lstm = calculate_mape(np.array(testdata['Predictions_lstm']))

## Plot prediction and true trends and log to Neptune         
# def plot_stock_trend_lstm(train, testdata):        
#     fig = plt.figure(figsize = (20,10))
#     plt.plot(train['Date'], train['price'], label = 'Train Closing Price')
#     # plt.plot(testdata['Date'], testdata['price'], label = 'testdata Closing Price')
#     plt.plot(testdata['Date'], testdata['Predictions_lstm'], label = 'Predicted Closing Price')
#     plt.title('LSTM Model')
#     plt.xlabel('Date')
#     plt.ylabel('Stock Price ($)')
#     plt.legend(loc="upper left")    
# plot_stock_trend_lstm(traindata, testdata)


,Stock,current_liabilities,total_revenue,current_Ratio,dividend_Payout,proprietary_ratio,Predictions_lstm
Date,,,,,,,
2019-01-31,Stock 1,834.292436,101.282767,0.881437,0.280577,0.346465,94.233566
2019-02-28,Stock 1,447.906629,119.483578,1.031850,0.221432,0.643166,94.866661
2019-03-29,Stock 1,683.167288,482.381171,0.774725,-0.000000,0.428754,93.434875
2019-04-30,Stock 1,350.779135,570.934617,1.176724,0.254717,0.625610,94.091591
2019-05-31,Stock 1,892.693784,277.522337,0.899257,-0.000000,0.711293,95.242966
...,...,...,...,...,...,...,...
2022-10-31,Stock 100,312.827547,150.677789,1.112762,-0.000000,0.518722,102.390556
2022-11-30,Stock 100,892.471680,529.905187,1.025366,0.208471,0.597438,101.421379
2022-12-30,Stock 100,420.021023,963.856236,0.841474,-0.000000,0.642605,102.026794
